## Generate a list of shops location data

Use Foursquare to get shops coordinates

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Foursquare ID info

In [2]:
CLIENT_ID = '5P4U2KHYRYF4YZIKIGSGT45I2ILKGBKE5TKNH2LIDZAPLO3J' # your Foursquare ID
CLIENT_SECRET = 'EF2SFFD0UYUVDKCLI0BVKPCTZGR50BMKTE5KTNQIUF4CWZHB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100

Foursquare only get 50 results for each enquiry. In order to get more McDonald's coordinates, search McDonald's near each MTR station.

In [3]:
mtr = pd.read_excel('mtr.xlsx', index_col=0)

In [4]:
# create a function to get location data
def get_locations(search_query, latitude, longitude, radius = 1000):

    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

    results = requests.get(url).json()

    # assign relevant part of JSON to venues
    venues = results['response']['venues']
    # tranform venues into a dataframe
    dataframe = json_normalize(venues)

    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]

    # function that extracts the category of the venue
    def get_category_type(row):
        try:
            categories_list = row['categories']
        except:
            categories_list = row['venue.categories']

        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']

    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

    for i, row in dataframe_filtered.iterrows():
        if (search_query in row['name']) == False:
            dataframe_filtered = dataframe_filtered.drop(i)
    
    return(dataframe_filtered)



In [5]:
loc_list = pd.DataFrame()

# loop for all stations and get the McDonald's shops around 5000 meters of the stations
for i, row in mtr.iterrows():
    s = row['Name']
    lat = row['Latitude']
    long = row['Longitude']
    loc_list = loc_list.append(get_locations('McDonald\'s',lat,long,5000), ignore_index=True, sort=False)
    

In [6]:
loc_list.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,state,id,postalCode
0,McDonald's Next (麥當勞),Fast Food Restaurant,"Portion A of Shop 1-41, G/F, Admiralty Centre,...",HK,金鐘,香港,NaN,56,"[Portion A of Shop 1-41, G/F, Admiralty Centre...","[{'label': 'display', 'lat': 22.27918285897123...",22.279183,114.164967,Central,中西區,56862e40498e9cd564f3707f,NaN
1,McDonald's (麥當勞),Fast Food Restaurant,"B/F, Yu To Sang Bldg, 37 Queen's Rd C",HK,香港,香港,NaN,949,"[B/F, Yu To Sang Bldg, 37 Queen's Rd C, 香港]","[{'label': 'display', 'lat': 22.2823055858633,...",22.282306,114.156039,NaN,NaN,4c777966d06b224b2897fed6,NaN
2,McDonald's (麥當勞),Fast Food Restaurant,"Shop G7A-B, G/F, Bank of America Tower, 12 Har...",HK,金鐘,香港,NaN,194,"[Shop G7A-B, G/F, Bank of America Tower, 12 Ha...","[{'label': 'display', 'lat': 22.28050873878890...",22.280509,114.163699,Central,中西區,4f502d75e4b0b71c78b7485a,NaN
3,McDonald's (麥當勞),Fast Food Restaurant,"2/F, 151 Lockhart Rd",HK,香港,香港,NaN,913,"[2/F, 151 Lockhart Rd, 香港]","[{'label': 'display', 'lat': 22.27827230150569...",22.278272,114.173339,Wan Chai,NaN,4bd80b0b0b779c740e4106a0,NaN
4,McDonald's (麥當勞),Fast Food Restaurant,"Shop 2022, 2/F, ifc mall, 8 Finance St",HK,中環,香港,NaN,992,"[Shop 2022, 2/F, ifc mall, 8 Finance St, 中西區, 香港]","[{'label': 'display', 'lat': 22.28518259658915...",22.285183,114.157639,NaN,中西區,4b2b1d94f964a52080b424e3,NaN


Group by the same ID so that we can remove all dupicate data

In [7]:
grouped = loc_list.groupby(['id'])

In [8]:
loc_list1 = pd.DataFrame()

for i, e in grouped:
    loc_list1 = loc_list1.append(e.iloc[0])
    
loc_list1

,address,categories,cc,city,country,crossStreet,distance,formattedAddress,id,labeledLatLngs,lat,lng,name,neighborhood,postalCode,state
31,"G/F-1/F, King's Commercial Centre, 25 King's Rd",Fast Food Restaurant,HK,香港,香港,NaN,2934.0,"[G/F-1/F, King's Commercial Centre, 25 King's ...",4b148822f964a5204ca423e3,"[{'label': 'display', 'lat': 22.28376202736191...",22.283762,114.192503,McDonald's (麥當勞),Causeway Bay,NaN,NaN
5,"Shop B, C & D4, G/F, Southorn Centre, 2 O'Brie...",Fast Food Restaurant,HK,香港,香港,NaN,919.0,"[Shop B, C & D4, G/F, Southorn Centre, 2 O'Bri...",4b16737ff964a5203cb923e3,"[{'label': 'display', 'lat': 22.27753396966267...",22.277534,114.173299,McDonald's (麥當勞),NaN,NaN,NaN
79,"Unit 32-37, 2/F, Langham Place, 8 Argyle St",Fast Food Restaurant,HK,旺角,香港,NaN,1508.0,"[Unit 32-37, 2/F, Langham Place, 8 Argyle St, 香港]",4b1913d5f964a520b5d723e3,"[{'label': 'display', 'lat': 22.31873479757754...",22.318735,114.168556,McDonald's (麥當勞),NaN,NaN,NaN
33,"Shop 5, B/F & G/F, Roca Centre, 460-470 King's Rd",Fast Food Restaurant,HK,北角,香港,NaN,3973.0,"[Shop 5, B/F & G/F, Roca Centre, 460-470 King'...",4b1a48b2f964a520a3e823e3,"[{'label': 'display', 'lat': 22.29115630930154...",22.291156,114.200735,McDonald's (麥當勞),NaN,NaN,NaN
43,"Shop G7A & B12-14, B1/F-G/F, Treasure World, W...",Fast Food Restaurant,HK,红磡,香港,NaN,3901.0,"[Shop G7A & B12-14, B1/F-G/F, Treasure World, ...",4b1b3f8ff964a52021fa23e3,"[{'label': 'display', 'lat': 22.30478211653743...",22.304782,114.190063,McDonald's (麥當勞),Hung Hom,NaN,NaN
26,"B/F, Pak Shing Bldg, 31-37 Jordan Rd",Fast Food Restaurant,HK,油麻地,香港,at Parkes St,3002.0,"[B/F, Pak Shing Bldg, 31-37 Jordan Rd (at Park...",4b20ce9bf964a520bc3424e3,"[{'label': 'display', 'lat': 22.30529849590855...",22.305298,114.170570,McDonald's (麥當勞),NaN,NaN,NaN
545,"Shop 304, LG/F, Lee On Shopping Centre, 23 Sha...",Fast Food Restaurant,HK,Ma On Shan Tsuen,香港,NaN,1620.0,"[Shop 304, LG/F, Lee On Shopping Centre, 23 Sh...",4b246677f964a520156724e3,"[{'label': 'display', 'lat': 22.42707509652946...",22.427075,114.240986,McDonald's (麥當勞),NaN,NaN,沙田區
142,"Unit 21, 1/F, Telford Plaza 1, 33 Wai Yip St",Fast Food Restaurant,HK,Kowloon Bay,香港,NaN,5724.0,"[Unit 21, 1/F, Telford Plaza 1, 33 Wai Yip St,...",4b24a84cf964a520416924e3,"[{'label': 'display', 'lat': 22.32420202850897...",22.324202,114.213621,McDonald's (麥當勞),NaN,NaN,NaN
4,"Shop 2022, 2/F, ifc mall, 8 Finance St",Fast Food Restaurant,HK,中環,香港,NaN,992.0,"[Shop 2022, 2/F, ifc mall, 8 Finance St, 中西區, 香港]",4b2b1d94f964a52080b424e3,"[{'label': 'display', 'lat': 22.28518259658915...",22.285183,114.157639,McDonald's (麥當勞),NaN,NaN,中西區
854,"Shop 9-10, G/F, Sing Shing Bldg, 90-114 Wo Yi ...",Fast Food Restaurant,HK,Kwai Chung,香港,NaN,1678.0,"[Shop 9-10, G/F, Sing Shing Bldg, 90-114 Wo Yi...",4b2f537ef964a520c9ea24e3,"[{'label': 'display', 'lat': 22.36785106368592...",22.367851,114.138934,McDonald's 麥當勞,NaN,NaN,NaN


Remove the McDonald's shops data not located in HK

In [9]:
loc_list1 = loc_list1[['id', 'name', 'lat', 'lng', 'address', 'formattedAddress', 'cc']]
loc_list1 = loc_list1[loc_list1['cc'] == 'HK']

Put the dataframe into an excel file

In [10]:
loc_list1.to_excel('m_shops.xlsx')

Use the same method to get Starbucks coordinates

In [11]:
loc_list = pd.DataFrame()

for i, row in mtr.iterrows():
    s = row['Name']
    lat = row['Latitude']
    long = row['Longitude']
    loc_list = loc_list.append(get_locations('Starbucks',lat,long,5000), ignore_index=True, sort=False)
 

In [12]:
grouped = loc_list.groupby(['id'])

In [13]:
loc_list1 = pd.DataFrame()

for i, e in grouped:
    loc_list1 = loc_list1.append(e.iloc[0])
    
loc_list1

,address,categories,cc,city,country,crossStreet,distance,formattedAddress,id,labeledLatLngs,lat,lng,name,neighborhood,postalCode,state
4,"Shop 201, 2/F, Alexandra House, 16-20 Chater Rd",Coffee Shop,HK,中環,香港,NaN,691.0,"[Shop 201, 2/F, Alexandra House, 16-20 Chater ...",4b05e882f964a52096e522e3,"[{'label': 'display', 'lat': 22.28166667371219...",22.281667,114.158478,Starbucks (星巴克),Central,NaN,中西區
135,"Shop 7-8, G/F, AIA Tower, 183 Electric Rd",Coffee Shop,HK,北角,香港,NaN,1177.0,"[Shop 7-8, G/F, AIA Tower, 183 Electric Rd, 香港]",4b0eeaf5f964a520c75c23e3,"[{'label': 'display', 'lat': 22.28808182193730...",22.288082,114.192481,Starbucks (星巴克),NaN,NaN,NaN
20,"Shop A, UG/F, Century Square, 1-13 D'Aguilar St",Coffee Shop,HK,香港,香港,NaN,911.0,"[Shop A, UG/F, Century Square, 1-13 D'Aguilar ...",4b109272f964a5200d7323e3,"[{'label': 'display', 'lat': 22.28162244504095...",22.281622,114.156140,Starbucks (星巴克),Central,NaN,NaN
102,"Shop 5, G/F, Palm Mansions, Whampoa Garden Site 4",Coffee Shop,HK,红磡,香港,at Tak Fung St,2331.0,"[Shop 5, G/F, Palm Mansions, Whampoa Garden Si...",4b10b4aef964a520f47423e3,"[{'label': 'display', 'lat': 22.30375824717172...",22.303758,114.189199,Starbucks (星巴克),NaN,NaN,NaN
228,"Shop G516, G/F, Kin On Mansion, Taikoo Shing, ...",Coffee Shop,HK,太古城,香港,NaN,3461.0,"[Shop G516, G/F, Kin On Mansion, Taikoo Shing,...",4b1332dcf964a520669523e3,"[{'label': 'display', 'lat': 22.28702800159346...",22.287028,114.213612,Starbucks (星巴克),Quarry Bay,NaN,NaN
7,"Shop M2, M/F, Baskerville House, 13 Duddell St",Coffee Shop,HK,中環,香港,NaN,783.0,"[Shop M2, M/F, Baskerville House, 13 Duddell S...",4b13a67ff964a520379823e3,"[{'label': 'display', 'lat': 22.27982617005152...",22.279826,114.156959,Starbucks (星巴克),Central,NaN,中西區
38,"Shop F1, 1/F, Fashion Walk, 27-47 Paterson St",Coffee Shop,HK,铜锣湾,香港,NaN,2158.0,"[Shop F1, 1/F, Fashion Walk, 27-47 Paterson St...",4b160a6ef964a52098b623e3,"[{'label': 'display', 'lat': 22.28050263206379...",22.280503,114.185381,Starbucks (星巴克),Causeway Bay,NaN,NaN
36,51 Caine Rd,Coffee Shop,HK,Mid-levels,香港,NaN,1249.0,"[51 Caine Rd, 香港]",4b16743ef964a52047b923e3,"[{'label': 'display', 'lat': 22.28085059955826...",22.280851,114.152552,Starbucks (星巴克),Mid-levels,NaN,NaN
28,"Shop B, G/F, Kwong Fat Hong Bldg, 1 Rumsey St",Coffee Shop,HK,Sheung Wan,香港,NaN,1391.0,"[Shop B, G/F, Kwong Fat Hong Bldg, 1 Rumsey St...",4b186280f964a520e7d123e3,"[{'label': 'display', 'lat': 22.28595087698068...",22.285951,114.153327,Starbucks (星巴克),NaN,NaN,NaN
19,"G/F, Fortis Bank Tower, 77-79 Gloucester Rd",Coffee Shop,HK,灣仔,香港,NaN,832.0,"[G/F, Fortis Bank Tower, 77-79 Gloucester Rd, ...",4b18f074f964a5209fd623e3,"[{'label': 'display', 'lat': 22.27952696798848...",22.279527,114.172550,Starbucks (星巴克),NaN,NaN,灣仔區


In [14]:
loc_list1 = loc_list1[['id', 'name', 'lat', 'lng', 'address', 'formattedAddress', 'cc']]
loc_list1 = loc_list1[loc_list1['cc'] == 'HK']
loc_list1.to_excel('s_shops.xlsx')